In [1]:
import pandas as pd
import os
os.chdir('/Users/theophile/Desktop/all')

In [2]:
train=pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train=train.drop(columns=["Name","Ticket","PassengerId"])
train['Sex']=train['Sex'].replace({'male' : 1, 'female' :0})
#train['Cabin'].fillna(train['Embarked'])

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,NaN,S
1,1,1,0,38.0,1,0,71.2833,C85,C
2,1,3,0,26.0,0,0,7.9250,NaN,S
3,1,1,0,35.0,1,0,53.1000,C123,S
4,0,3,1,35.0,0,0,8.0500,NaN,S


In [4]:
results=train['Survived']
inputs=train.drop(columns='Survived')
inputs.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,NaN,S
1,1,0,38.0,1,0,71.2833,C85,C
2,3,0,26.0,0,0,7.9250,NaN,S
3,1,0,35.0,1,0,53.1000,C123,S
4,3,1,35.0,0,0,8.0500,NaN,S


In [5]:
inputs['Cabin']=inputs['Cabin'].fillna(' ')
inputs['Cabin']=inputs['Cabin'].apply(lambda x: x[1:])
inputs['Cabin']=inputs['Cabin'].replace('',0)
inputs.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,0,S
1,1,0,38.0,1,0,71.2833,85,C
2,3,0,26.0,0,0,7.9250,0,S
3,1,0,35.0,1,0,53.1000,123,S
4,3,1,35.0,0,0,8.0500,0,S


In [6]:
inputs['Embarked']=inputs['Embarked'].replace('S',0)
inputs['Embarked']=inputs['Embarked'].replace('C',1)
inputs['Embarked']=inputs['Embarked'].replace('Q',2)
inputs.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.0,1,0,7.2500,0,0.0
1,1,0,38.0,1,0,71.2833,85,1.0
2,3,0,26.0,0,0,7.9250,0,0.0
3,1,0,35.0,1,0,53.1000,123,0.0
4,3,1,35.0,0,0,8.0500,0,0.0


In [7]:
import numpy as np 
def nb_val(col):
    return(sum(col.value_counts()))
L=[nb_val(inputs[c]) for c in inputs.columns]
print(L)

[891, 891, 714, 891, 891, 891, 891, 889]


In [8]:
train['Embarked']=train['Embarked'].replace('S',0)
train['Embarked']=train['Embarked'].replace('C',1)
train['Embarked']=train['Embarked'].replace('Q',2)

In [9]:
inputs=inputs[inputs['Embarked'] >= 0]
train=train[train['Embarked']>=0]
results=train['Survived']
L=[nb_val(inputs[c]) for c in inputs.columns]
print(L)

[889, 889, 712, 889, 889, 889, 889, 889]


In [10]:
inputs['Age']=inputs['Age'].fillna(inputs['Age'].mean())
L=[nb_val(inputs[c]) for c in inputs.columns]
print(L)

[889, 889, 889, 889, 889, 889, 889, 889]


In [11]:
import numpy as np
for c in ['Age','Fare']:
    inputs[c]=(lambda x:( x-x.mean() )/np.sqrt( x.var() ))(inputs[c])

print(inputs['Age'].mean())

8.058420822270807e-15


In [12]:
results.value_counts()

0    549
1    340
Name: Survived, dtype: int64

In [13]:
import sklearn
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [14]:
inputs.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,-0.589288,1,0,-0.499958,0,0.0
1,1,0,0.644485,1,0,0.788503,85,1.0
2,3,0,-0.280845,0,0,-0.486376,0,0.0
3,1,0,0.413153,1,0,0.422623,123,0.0
4,3,1,0.413153,0,0,-0.483861,0,0.0


In [15]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
inputs=inputs.drop(columns='Cabin')
lm.fit(inputs, results)
pred_res = lm.predict(inputs)
a=pred_res-results
a.value_counts()

 0    715
-1    105
 1     69
Name: Survived, dtype: int64

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(inputs, results, test_size = 0.33, random_state = 5)
lm = LogisticRegression()
lm.fit(X_train, Y_train)
Y_pred = lm.predict(X_test)
(Y_pred-Y_test).value_counts()

 0    244
-1     31
 1     19
Name: Survived, dtype: int64

Test du modèle de prédiction sur le csv test

In [17]:
inputs=pd.read_csv('test.csv')
inputs=inputs.drop(columns=["Name","Ticket","PassengerId"])
inputs['Sex']=inputs['Sex'].replace({'male' : 1, 'female' :0})
inputs['Cabin']=inputs['Cabin'].fillna(' ')
inputs['Cabin']=inputs['Cabin'].apply(lambda x: x[1:])
inputs['Cabin']=inputs['Cabin'].replace('',0)
inputs['Embarked']=inputs['Embarked'].replace('S',0)
inputs['Embarked']=inputs['Embarked'].replace('C',1)
inputs['Embarked']=inputs['Embarked'].replace('Q',2)
inputs=inputs[inputs['Embarked'] >= 0]
inputs['Age']=inputs['Age'].fillna(inputs['Age'].mean())
inputs['Fare']=inputs['Fare'].fillna(inputs['Fare'].mean())

import numpy as np
for c in ['Age','Fare']:
    inputs[c]=(lambda x:( x-x.mean() )/np.sqrt( x.var() ))(inputs[c])
inputs=inputs.drop(columns='Cabin')
pred_res = lm.predict(inputs)
